# Tutorial 0

## Generation of Example Data

This tutorial generates example data to be used in some of the other tutorials. The example data can be downloaded from [here](https://github.com/almoulla/arve/tree/main/docs/tutorials/example_data).

We begin by importing the `ARVE` package and other useful packages.

In [1]:
# Import packages
import arve
import numpy  as np
import pandas as pd

### Synthetic Stellar Spectral Time Series

In this part, we simulate a stellar spectral time series. The spectral time series consists of 100 observations each containing the same 100 spectral lines. 95% of the lines are displaced with a sinusoidal signal of 10 m/s amplitude, whereas the remaining 5% of the lines are randomly shifted by RVs drawn from a $\mathcal{G}(\mu=100\,\mathrm{m/s}, \sigma=100\,\mathrm{m/s})$ distribution acting as spurious outliers (from, e.g., telluric contamination). The spectra are thereafter noised to a S/N of 200.

In [2]:
# Parameters
N_spec = 100 # nr. of spectra
N_line = 100 # nr. of lines

# Constants
c = 2.99792458e5 # [km/s] speed of light in vacuum

# Initiate ARVE object
example = arve.ARVE()
example.star.target = "Sun"
example.star.get_stellar_parameters()
example.data.add_data(wave_val=np.arange(3000,3100,0.01).reshape(1,-1))
example.data.get_aux_data()

# Get line mask
mask = example.data.aux_data["mask"][0]
mask = mask[:N_line]
wc   = mask["wave" ].values
dc   = mask["depth"].values

# Wavelength grid
wave_min = wc[ 0] - 0.5
wave_max = wc[-1] + 0.5
wave_val = np.arange(wave_min, wave_max, 0.01)
N_wave   = len(wave_val)

# Good and bad lines
frac_good     = 0.95
N_line_good   = int(N_line*frac_good)
N_line_bad    = N_line - N_line_good
idx_line_good = np.sort(np.random.default_rng().choice(N_line, N_line_good, replace=False))

# RVs of good and bad lines
time_val      = np.arange(N_spec)
time_span     = time_val[-1] - time_val[0]
vrad_val_good = 10e-3*np.sin(2*np.pi*time_val/time_span)
vrad_val_bad  = np.random.normal(100e-3, 100e-3, N_spec)

# Flux matrix
flux_val = np.ones((N_spec, N_wave))

# Loop spectra
for i in range(N_spec):

    # Loop lines
    for j in range(N_line):

        # RV shift
        if j in idx_line_good:
            v = vrad_val_good[i]
        else:
            v = vrad_val_bad[i]

        # Shifted wavelength
        wc_shift = wc[j]*(1+v/c)

        # Line width in velocity
        sig_v = 3

        # Line width in wavelength
        sig_w = wc[j]*sig_v/c

        # Add line to spectrum
        flux_val[i] -= dc[j]*np.exp(-(wave_val-wc_shift)**2/(2*sig_w**2))

# Degrade spectrum
snr_val      = 200
cont_val_abs = snr_val**2
flux_val_abs = flux_val*cont_val_abs
flux_val_abs = np.random.poisson(flux_val_abs)
flux_err_abs = np.sqrt(flux_val_abs)
flux_val     = flux_val_abs/cont_val_abs
flux_err     = flux_err_abs/cont_val_abs

# Save spectra
for i in range(N_spec):
    np.savez(f"example_data/tutorial_2/spec_{i:0>2}.npz",
    time_val=time_val[i],
    wave_val=wave_val   ,
    flux_val=flux_val[i],
    flux_err=flux_err[i])

# Save RVs
df = pd.DataFrame()
df["time_val"     ] = time_val
df["vrad_val_good"] = vrad_val_good
df["vrad_val_bad" ] = vrad_val_bad
df.to_csv("example_data/tutorial_2/vrad.csv", index=False)

### Synthetic Radial Velocities

In this part, we use `ARVE` to simulate realistic radial velocities (RVs) of stellar oscillations and granulation. White noise is thereafter added to the RVs.

In [3]:
# Simulate RVs with ARVE
example = arve.ARVE()
example.star.target = "Sun"
example.star.get_stellar_parameters()
example.star.add_vpsd_components()
example.star.simulate_vrad_from_vpsd_components(time_start=0, time_stop=10, time_step=1/(24*60))

# Extract RVs from ARVE object and add white noise
time_val  = example.data.vrad_components["time"]
vrad_val  = example.data.vrad_components["total"]
vrad_err  = 1e-3
vrad_val += np.random.normal(0, vrad_err, len(time_val))
vrad_err  = np.ones_like(time_val)*vrad_err

# Save RVs in CSV file
df = pd.DataFrame()
df["time_val"] = time_val
df["vrad_val"] = vrad_val
df["vrad_err"] = vrad_err
df.to_csv("example_data/tutorial_3/vrad.csv", index=False)